In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np,gc 
import pandas as pd 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', 500)

In [2]:
categorical_type=['ProductCD','card4','card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6', 'M7', 'M8', 'M9','id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo','id-12', 'id-15', 'id-16', 'id-23', 'id-27', 'id-28', 'id-29', 'id-30', 
            'id-31', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38']
cols=['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain']
cols+=['C'+str(i) for i in range(1,15)]
cols+=['D'+str(i) for i in range(1,16)]
cols+=['M'+str(i) for i in range(1,10)]
v=[1, 3, 4, 6, 8, 11]
v+=[13, 14, 16, 17, 20, 23, 26, 27, 30]
v+=[36, 37, 40, 41, 44, 45, 47, 48, 52]
v+=[76, 78, 80, 82, 86, 88, 89, 91]
v+=[127, 98, 99, 107, 108, 111, 115, 117, 121, 123, 130, 136]
v+=[138, 139, 142, 147, 155, 158, 162]
v+=[165, 160]
v+=[203, 207, 173, 206, 215]
v+=[169, 171, 175, 209, 185, 188, 198, 210]
v+=[218, 223, 224, 226, 228, 229, 235]
v += [240, 258, 257, 253, 252, 260, 261]
v += [264, 266, 267, 274, 277]
v += [220, 221, 234, 238, 250, 271]
v += [294, 284, 285, 286, 291, 297]
v += [303, 305, 307, 309, 310, 320]
v += [281, 283, 289, 296, 301, 314]
#v += [332, 325, 335, 338]
cols+=['V'+str(i) for i in v]
dtypes = {}
for index in cols+['id_0'+str(i) for i in range(1,10)]+['id_'+str(i) for i in range(10,34)]+\
    ['id-0'+str(i) for i in range(1,10)]+['id-'+str(i) for i in range(10,34)]:
        dtypes[index] = 'float32'
for index in categorical_type: dtypes[index] = 'category'

In [3]:
train = pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv',index_col='TransactionID', dtype=dtypes, usecols=cols+['isFraud'])
train_id = pd.read_csv('../input/ieee-fraud-detection/train_identity.csv',index_col='TransactionID', dtype=dtypes)
train = train.merge(train_id, how='left', left_index=True, right_index=True)

test = pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv',index_col='TransactionID', dtype=dtypes, usecols=cols)
test_id = pd.read_csv('../input/ieee-fraud-detection/test_identity.csv',index_col='TransactionID', dtype=dtypes)
fix = {o:n for o, n in zip(test_id.columns, train_id.columns)}
test_id.rename(columns=fix, inplace=True)
test = test.merge(test_id, how='left', left_index=True, right_index=True)
# TARGET
y = train['isFraud'].copy()
del train_id,test_id,train['isFraud']; x = gc.collect()

print('Train shape',train.shape,'test shape',test.shape)

Train shape (590540, 202) test shape (506691, 202)


In [4]:
train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V14,V16,V17,V20,V23,V26,V27,V30,V36,V37,V40,V41,V44,V45,V47,V48,V52,V76,V78,V80,V82,V86,V88,V89,V91,V98,V99,V107,V108,V111,V115,V117,V121,V123,V127,V130,V136,V138,V139,V142,V147,V155,V158,V160,V162,V165,V169,V171,V173,V175,V185,V188,V198,V203,V206,V207,V209,V210,V215,V218,V220,V221,V223,V224,V226,V228,V229,V234,V235,V238,V240,V250,V252,V253,V257,V258,V260,V261,V264,V266,V267,V271,V274,V277,V281,V283,V284,V285,V286,V289,V291,V294,V296,V297,V301,V303,V305,V307,V309,V310,V314,V320,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000.0,86400.0,68.5,W,13926.0,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,117.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001.0,86401.0,29.0,W,2755.0,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002.0,86469.0,59.0,W,4663.0,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,T,T,T,M0,F,F,F,F,F,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003.0,86499.0,50.0,W,18132.0,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN,NaN

In [5]:
test.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V14,V16,V17,V20,V23,V26,V27,V30,V36,V37,V40,V41,V44,V45,V47,V48,V52,V76,V78,V80,V82,V86,V88,V89,V91,V98,V99,V107,V108,V111,V115,V117,V121,V123,V127,V130,V136,V138,V139,V142,V147,V155,V158,V160,V162,V165,V169,V171,V173,V175,V185,V188,V198,V203,V206,V207,V209,V210,V215,V218,V220,V221,V223,V224,V226,V228,V229,V234,V235,V238,V240,V250,V252,V253,V257,V258,V260,V261,V264,V266,V267,V271,V274,V277,V281,V283,V284,V285,V286,V289,V291,V294,V296,V297,V301,V303,V305,V307,V309,V310,V314,V320,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549.0,18403224.0,31.950001,W,10409.0,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,T,T,F,NaN,NaN,F,T,T,T,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,47.950001,47.950001,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,47.950001,0.000000,47.950001,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663550.0,18403264.0,49.000000,W,4272.0,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,NaN,231.0,634.0,NaN,NaN,NaN,634.0,T,F,F,M0,NaN,F,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,280.000000,280.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,280.000000,0.000000,280.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663551.0,18403310.0,171.000000,W,4476.0,574.0,150.0,visa,226.0,debit,472.0,87.0,2635.0,NaN,hotmail.com,NaN,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,NaN,NaN,NaN,NaN,136.0,136.0,NaN,NaN,NaN,97.0,T,T,F,M0,F,F,F,F,F,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,968.000000,705.000000,263.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,1321.000000,0.000000,1058.000000,0.000000,263.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663552.0,18403310.0,284.950012,W,10989.0,360.0,150.0,visa,166.0,debit,205.0,87.0,17.0,NaN,gmail.com,N

In [6]:
for i in range(1,16):
    if i in [1,2,3,5,9]:continue
    train['D'+str(i)]= train['D'+str(i)]-train['TransactionDT']/np.float32(24*60*60)
    test['D'+str(i)] = test['D'+str(i)] - test['TransactionDT']/np.float32(24*60*60)

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(train['TransactionDT'],train['D9'])
plt.show()

In [7]:
ids=['id_0'+str(i) for i in range(1,10)]+['id_'+str(i) for i in range(10,39)]
for column in ids:
    if train[column].isna().sum()>500000:
        train.drop([column],axis=1,inplace=True)

In [8]:
train.shape

(590540, 183)

In [9]:
train.shape

(590540, 183)

In [10]:
for index,column in enumerate(train.columns):
    if (np.str(train[column].dtype)=='category')|(np.str(train[column].dtype)=='object'):
        train_test=pd.concat([train[column],test[column]],axis=0)
        train_test,_=train_test.factorize(sort=True)
        train[column] = train_test[:len(train)].astype('int16')
        test[column] = train_test[len(train):].astype('int16')

    elif column not in ['TransactionAmt','TransactionDT']:
        #mn =train[column].min()
        #train[column] -= np.float32(mn)
        train[column].fillna(-1,inplace=True)
        test[column].fillna(-1,inplace=True)
        

In [11]:
train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V14,V16,V17,V20,V23,V26,V27,V30,V36,V37,V40,V41,V44,V45,V47,V48,V52,V76,V78,V80,V82,V86,V88,V89,V91,V98,V99,V107,V108,V111,V115,V117,V121,V123,V127,V130,V136,V138,V139,V142,V147,V155,V158,V160,V162,V165,V169,V171,V173,V175,V185,V188,V198,V203,V206,V207,V209,V210,V215,V218,V220,V221,V223,V224,V226,V228,V229,V234,V235,V238,V240,V250,V252,V253,V257,V258,V260,V261,V264,V266,V267,V271,V274,V277,V281,V283,V284,V285,V286,V289,V291,V294,V296,V297,V301,V303,V305,V307,V309,V310,V314,V320,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000.0,86400.0,68.5,4,13926.0,-1.0,150.0,1,142.0,1,315.0,87.0,19.0,-1.0,-1,-1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,-1.0,13.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,12.000000,12.000000,-1.0,-1.0,-1.0,-1.000000,0,1,1,2,0,1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,117.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,117.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1,-1,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1
2987001.0,86401.0,29.0,4,2755.0,404.0,150.0,2,102.0,1,325.0,87.0,-1.0,-1.0,16,-1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.000012,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000012,-1.000000,-1.0,-1.0,-1.0,-1.000012,-1,-1,-1,0,1,1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1,-1,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1
2987002.0,86469.0,59.0,4,4663.0,490.0,150.0,3,166.0,2,330.0,87.0,287.0,-1.0,35,-1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.000799,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000799,313.999207,-1.0,-1.0,-1.0,313.999207,0,1,1,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1,-1,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1
2987003.0,86499.0,50.0,4,18132.0,567.0,150.0,2,117.0,2,476.0,87.0,-1.0,-1.0,54,-1,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,92.998856,0.0,-1.0,-1.0,-1.0,-1.0,82.998856,-1.000000,-1.0,-1.0,-1.0,109.998856,-1,-1,-1,0,1,0,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,10.0,1.0,1.0,1.0,1.0,1.0,

In [12]:
test.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V14,V16,V17,V20,V23,V26,V27,V30,V36,V37,V40,V41,V44,V45,V47,V48,V52,V76,V78,V80,V82,V86,V88,V89,V91,V98,V99,V107,V108,V111,V115,V117,V121,V123,V127,V130,V136,V138,V139,V142,V147,V155,V158,V160,V162,V165,V169,V171,V173,V175,V185,V188,V198,V203,V206,V207,V209,V210,V215,V218,V220,V221,V223,V224,V226,V228,V229,V234,V235,V238,V240,V250,V252,V253,V257,V258,V260,V261,V264,V266,V267,V271,V274,V277,V281,V283,V284,V285,V286,V289,V291,V294,V296,V297,V301,V303,V305,V307,V309,V310,V314,V320,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549.0,18403224.0,31.950001,4,10409.0,111.0,150.0,3,226.0,2,170.0,87.0,1.0,-1.0,16,-1,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,184.999725,27.0,-1.0,-1.0,-1.0,-1.0,204.999725,-10.000275,-1.0,-1.0,-1.0,195.999725,0,1,0,-1,-1,0,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,47.950001,47.950001,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,47.950001,0.000000,47.950001,0.000000,0.0,-1.0,-1.0,NaN,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1.0,-1,-1.0,NaN,-1,-1,-1.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,NaN,-1,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1
3663550.0,18403264.0,49.000000,4,4272.0,111.0,150.0,3,226.0,2,299.0,87.0,4.0,-1.0,2,-1,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,420.999268,7.0,-1.0,-1.0,-1.0,-1.0,17.999252,420.999268,-1.0,-1.0,-1.0,420.999268,0,0,0,0,-1,0,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,280.000000,280.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,280.000000,0.000000,280.000000,0.000000,0.0,-1.0,-1.0,NaN,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1.0,-1,-1.0,NaN,-1,-1,-1.0,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,NaN,-1,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1
3663551.0,18403310.0,171.000000,4,4476.0,574.0,150.0,3,226.0,2,472.0,87.0,2635.0,-1.0,19,-1,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,-116.001266,10.0,-1.0,-1.0,-1.0,-1.0,-77.001266,-77.001266,-1.0,-1.0,-1.0,-116.001266,0,1,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,968.000000,705.000000,263.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,1321.000000,0.000000,1058.000000,0.000000,263.0,-1.0,-1.0,NaN,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1.0,-1,-1.0,NaN,-1,-1,-1.0,NaN,-1.0,-1.0,NaN,NaN,

In [13]:
cols = list( train.columns )
cols.remove('TransactionDT')

In [14]:

for c in ['D6','D7','D8','D9','D12','D13','D14']:
    cols.remove(c)

In [15]:
len(cols)
np.array(cols)

array(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4',
       'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2',
       'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5',
       'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1',
       'D2', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15', 'M1', 'M2', 'M3',
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V3', 'V4', 'V6', 'V8',
       'V11', 'V13', 'V14', 'V16', 'V17', 'V20', 'V23', 'V26', 'V27',
       'V30', 'V36', 'V37', 'V40', 'V41', 'V44', 'V45', 'V47', 'V48',
       'V52', 'V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89', 'V91',
       'V98', 'V99', 'V107', 'V108', 'V111', 'V115', 'V117', 'V121',
       'V123', 'V127', 'V130', 'V136', 'V138', 'V139', 'V142', 'V147',
       'V155', 'V158', 'V160', 'V162', 'V165', 'V169', 'V171', 'V173',
       'V175', 'V185', 'V188', 'V198', 'V203', 'V206', 'V207', 'V209',
       'V210', 'V215', 'V218', 'V220', 'V221', 'V223', 'V224', 'V226',
       'V228', '

In [16]:
import datetime
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
train['DT_M'] = train['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
train['DT_M'] = (train['DT_M'].dt.year-2017)*12 + train['DT_M'].dt.month 

test['DT_M'] = test['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
test['DT_M'] = (test['DT_M'].dt.year-2017)*12 + test['DT_M'].dt.month 

In [17]:
import xgboost as xgb
preds=np.zeros(len(test))

fold=GroupKFold(n_splits=6)
for i, (index_train,index_valid) in enumerate(fold.split(train,y,groups=train['DT_M'])):
    month=train.iloc[index_valid]['DT_M'].iloc[0]
    print('Fold',i,'withholding month',month)
    clf=xgb.XGBClassifier(
        n_estimators=5000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        nthread=4,
        tree_method='hist')
    h = clf.fit(train[cols].iloc[index_train], y.iloc[index_train], 
            eval_set=[(train[cols].iloc[index_valid],y.iloc[index_valid])],
            verbose=100, early_stopping_rounds=200)
    preds += clf.predict_proba(test[cols])[:,1]/fold.n_splits

Fold 0 withholding month 12


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.82045
[100]	validation_0-auc:0.86476
[200]	validation_0-auc:0.88047
[300]	validation_0-auc:0.89594
[400]	validation_0-auc:0.90392
[500]	validation_0-auc:0.90676
[600]	validation_0-auc:0.90781
[700]	validation_0-auc:0.90821
[800]	validation_0-auc:0.90832
[900]	validation_0-auc:0.90766
[952]	validation_0-auc:0.90787


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Fold 1 withholding month 15
[0]	validation_0-auc:0.84358
[100]	validation_0-auc:0.89436
[200]	validation_0-auc:0.91405
[300]	validation_0-auc:0.92855
[400]	validation_0-auc:0.93538
[500]	validation_0-auc:0.93747
[600]	validation_0-auc:0.93829
[700]	validation_0-auc:0.93872
[800]	validation_0-auc:0.93897
[900]	validation_0-auc:0.93895
[1000]	validation_0-auc:0.93839
[1027]	validation_0-auc:0.93846
Fold 2 withholding month 13
[0]	validation_0-auc:0.84008
[100]	validation_0-auc:0.89203
[200]	validation_0-auc:0.90870
[300]	validation_0-auc:0.92421
[400]	validation_0-auc:0.93261
[500]	validation_0-auc:0.93567
[600]	validation_0-auc:0.93760
[700]	validation_0-auc:0.93810
[800]	validation_0-auc:0.93841
[900]	validation_0-auc:0.93846
[1000]	validation_0-auc:0.93873
[1100]	validation_0-auc:0.93828
[1178]	validation_0-auc:0.93829
Fold 3 withholding month 17
[0]	validation_0-auc:0.83553
[100]	validation_0-auc:0.88896
[200]	validation_0-auc:0.91054
[300]	validation_0-auc:0.92520
[400]	validation_0

In [ ]:
import xgboost as xgb
clf=xgb.XGBClassifier(
    n_estimators=2000,
    max_depth=12, 
    learning_rate=0.02, 
    subsample=0.8,
    colsample_bytree=0.4, 
    missing=-1, 
    eval_metric='auc')
history = clf.fit(train.loc[index_train,cols], y[index_train], 
        eval_set=[(train.loc[index_valid,cols],y[index_valid])],
        verbose=50, early_stopping_rounds=100)


In [ ]:
preds=clf.predict_proba(test[cols])[:,1]

In [18]:
len(preds)

506691

In [19]:
sample_submission=pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')
sample_submission['isFraud']=preds


In [20]:
sample_submission.to_csv('xgb_sub3.csv',index=False)